# Research Question 2 

## What are consumers issues with the current system as well as their concerns about the transition towards renewable energy

This analysis focuses on understanding consumer concerns regarding the current energy system and their perspectives on transitioning to renewable energy, using the 'Impact Of Transition' and 'Concern for Ability to Pay' data from the Australia ECSS dataset.

#### Importing Necessary Libraries

In [11]:
import pandas as pd
import numpy as np
#import chart_studio.plotly as py
#import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import textwrap
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#### Initialize Plotly and Cufflinks for interactive visualizations

In [12]:
init_notebook_mode(connected=True)
cf.go_offline()

NameError: name 'cf' is not defined

#### Loading the Data from Excel File

In [4]:
# Load the entire Excel file once to avoid redundant loading later
file_path = '../Dataset/ecss-jun-24-topline-data.xlsx'
data = pd.read_excel(file_path, sheet_name=None)  # Load all sheets into a dictionary

# Display sheet names to ensure they are loaded correctly
print(data.keys())


dict_keys(['TOC', 'Satisfaction with service', 'Value for money', 'Satisfaction with electricity r', 'Outages', 'Satisfaction with service(2)', 'Value for money(2)', 'Satisfaction with gas retailer', 'Switching behaviour', 'Reasons for switching', 'Frequency of investigating swit', 'Remembering communication', 'Attention', 'Concern for ability to pay', 'Useful & Easy to read (New)', 'Reasons for higher bills', 'Electricity bill support', 'Confidence in abilities and sup', 'Confidence in overall market', 'Confidence in future outcomes', 'Awareness of ombudsman', 'Value for money of utilities', 'Concern with paying bills', 'Trust in utilities', 'Challenges ahead for the energy', 'Energy consumer service prefere', 'Uptake of solar PV panels', 'Steps to adjust to the transiti', 'Impact of transition', 'Communication of the transition', 'Satisfaction with service(3)', 'Value for money(3)', 'Satisfaction with electricit(2)', 'Outages(2)', 'Cost to business', 'Satisfaction with service(4)', '

#### Viweing The First Few Rows Of Impact Of transition

In [5]:
# View the first few rows of the "Impact of transition" sheet to understand its structure
print(data['Impact of transition'].head())


                                         Back to TOC           Unnamed: 1  \
0  T9: How do you think the transition to renewab...                  NaN   
1                                                NaN  Household Lifestage   
2                                           Column %       All households   
3  Electricity will become cheaper as moving to r...             0.118517   
4  Overall, the transition will not affect electr...              0.07747   

        Unnamed: 2        Unnamed: 3       Unnamed: 4         Unnamed: 5  \
0              NaN               NaN              NaN                NaN   
1              NaN               NaN              NaN                NaN   
2  Young Household  Family Household  Older Household  Retired Household   
3         0.215411          0.130136         0.112319           0.088611   
4          0.11246          0.066223          0.07009           0.079316   

  Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...   Unnamed: 15 Unnamed: 16  \


## Visualizing Consumer Concerns About the Transition to Renewable Energy

We will now analyze and visualize the data from the Impact of Transition sheet.

#### Cleaning and Preprocessing Data for Visualization

In [6]:
# Define the wrap_text function to wrap long text labels
def wrap_text(text, width=40):
    return "<br>".join(textwrap.wrap(text, width=width))

# Clean and rename the columns in the "Impact of Transition" sheet for readability and ease of analysis
impact_transition_data = data['Impact of transition'].copy()

# Rename the columns based on the actual structure (relevant columns for States and Column %)
impact_transition_data.columns = ['Statement', 'All households', 'Young Household', 'Family Household', 
                                  'Older Household', 'Retired Household', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 
                                  'TAS', 'ACT', 'Inner Metro', 'Outer Metro', 'Country Town', 'Rural', 
                                  'Financially Comfortable', 'Struggle to Afford', 'Under Financial Pressure', 
                                  'Homeowner', 'Renter', 'House', 'Townhouse or Duplex', 'Unit, Flat, Apartment']

# Convert the 'Statement' column to string first, filling any NaN values with empty strings
impact_transition_data['Statement'] = impact_transition_data['Statement'].fillna('').astype(str)

# Now remove irrelevant rows such as "NET", "Column n", and "Column %" from the 'Statement' column
impact_transition_data = impact_transition_data[
    (impact_transition_data['Statement'] != 'NET') & 
    (impact_transition_data['Statement'] != 'Column n') & 
    (~impact_transition_data['Statement'].str.contains('Column %'))
]


# Ensure that the 'Statement' column contains only strings by converting any non-string data to an empty string
impact_transition_data['Statement'] = impact_transition_data['Statement'].fillna('').astype(str)

# Apply text wrapping to longer statements for better display in the charts
impact_transition_data['Statement'] = impact_transition_data['Statement'].apply(lambda x: wrap_text(x))

# Shorten some response statements for simpler display
def shorten_statement(statement):
    if "expensive" in statement:
        return "Electricity more expensive"
    elif "cheaper" in statement:
        return "Electricity cheaper"
    elif "affect prices" in statement:
        return "No price effect"
    elif "Unsure" in statement:
        return "Unsure about transition"
    return statement

impact_transition_data['Statement'] = impact_transition_data['Statement'].apply(shorten_statement)

# Focus ONLY on the relevant state columns and questions for visualization
states_to_visualize = impact_transition_data[['Statement', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'ACT']].dropna()


### Visualization: Public Sentiment on Renewable Energy Transition

In [8]:
# Ensure that all state columns are numeric (convert strings or NaNs to numeric, with invalid values turned into NaN)
for state in ['NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'ACT']:
    states_to_visualize[state] = pd.to_numeric(states_to_visualize[state], errors='coerce')

# Define custom colors for each state
colors = {
    'NSW': '#636EFA', 
    'VIC': '#EF553B', 
    'QLD': '#00CC96', 
    'WA': '#AB63FA', 
    'SA': '#FFA15A', 
    'TAS': '#19D3F3', 
    'ACT': '#FF6692'
}

# Create the interactive figure using dropdowns
fig = go.Figure()

# Create a sorted bar chart for each state
for state in ['NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'ACT']:
    sorted_data = states_to_visualize.sort_values(by=state, ascending=False)
    fig.add_trace(go.Bar(y=sorted_data['Statement'],
                         x=sorted_data[state],
                         text=sorted_data[state].apply(lambda x: f'{x:.1%}' if pd.notna(x) else ''),  # Format values as percentages
                         textposition='auto',
                         marker_color=colors[state],
                         orientation='h',  # Horizontal bar chart
                         name=state,
                         visible=False))

# Set the initial visible state to NSW
fig.data[0].visible = True

# Create the dropdown menu for state selection
dropdown_buttons = []
for i, state in enumerate(['VIC', 'NSW', 'QLD', 'WA', 'SA', 'TAS', 'ACT']):
    dropdown_buttons.append(
        dict(
            method='update',
            label=state,
            args=[{'visible': [j == i for j in range(len(fig.data))]},  # Set visibility for the selected state
                  {'title': f'Public Sentiment on Renewable Energy Transition in {state}'}]
        )
    )

# Update the layout with a centered title and other improvements
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        x=1.15,
        y=1.15
    )],
    title={
        'text': "Public Sentiment on Renewable Energy Transition by State",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    yaxis_title="Public Opinions",
    xaxis_title="Percentage of Respondents (%)",
    xaxis_tickformat=".0%",
    showlegend=False
)

# Display the interactive figure
fig.show()
fig.write_json("Public_outlook_renewable_by_state")

#### **Summary:**
The visualization above shows that a large percentage of consumers believe that transitioning to renewable energy will make electricity more expensive, with high agreement levels in several states.


## Visualizing Concerns About People's Ability to Pay

Now we will clean and visualize data from the Concern for Ability to Pay sheet using a heatmap.

#### Cleaning and Preprocessing Data for Visualization

In [9]:
# Load the 'Concern for Ability to Pay' sheet and clean it
concern_for_ability_to_pay = data['Concern for ability to pay'].copy()

# Clean and rename headers based on actual data structure
headers = concern_for_ability_to_pay.iloc[2]
concern_for_ability_to_pay_cleaned = concern_for_ability_to_pay.drop([0, 1, 2]).reset_index(drop=True)
concern_for_ability_to_pay_cleaned.columns = headers

# Drop fully empty columns and convert relevant columns to numeric
concern_for_ability_to_pay_cleaned = concern_for_ability_to_pay_cleaned.dropna(how='all', axis=1)
concern_for_ability_to_pay_cleaned = concern_for_ability_to_pay_cleaned.apply(pd.to_numeric, errors='ignore')

# Inspect the cleaned data
print(concern_for_ability_to_pay_cleaned.head())


2                    Column %  All households  Young Household  \
0              Strongly agree        0.211570         0.263780   
1              Somewhat agree        0.277599         0.381896   
2  Neither agree nor disagree        0.223297         0.190366   
3           Somewhat disagree        0.134297         0.081070   
4           Strongly disagree        0.144854         0.061740   

2  Family Household  Older Household  Retired Household       NSW     VIC  \
0          0.305283         0.229146           0.129591  0.226933  0.2100   
1          0.340967         0.282863           0.207261  0.284289  0.2750   
2          0.200096         0.225980           0.244328  0.206983  0.2325   
3          0.080487         0.126662           0.186232  0.122195  0.1525   
4          0.071271         0.127033           0.223973  0.147132  0.1275   

2       QLD        WA  ...  Country Town     Rural  Financially comfortable  \
0  0.226667  0.203333  ...      0.194710  0.248220           

### Heatmap of Concern Levels

In [14]:
# Filter out irrelevant rows for heatmap visualization
filtered_ability_to_pay_data = concern_for_ability_to_pay_cleaned[
    ~concern_for_ability_to_pay_cleaned['Column %'].isin(['NET', 'Column n'])
]

# Prepare data for heatmap by melting it
melted_ability_to_pay_data = filtered_ability_to_pay_data.melt(
    id_vars=["Column %"], 
    value_vars=['All households', 'Young Household', 'Family Household', 
                'Older Household', 'Retired Household', 'NSW', 'VIC', 'QLD', 'WA'],
    var_name='Category',
    value_name='Concern Level'
)

# Convert concern levels to percentages
melted_ability_to_pay_data['Concern Level'] = melted_ability_to_pay_data['Concern Level'] * 100

# Create the heatmap
fig = px.density_heatmap(
    melted_ability_to_pay_data, 
    x='Category', 
    y='Column %', 
    z='Concern Level',
    color_continuous_scale='Blues',
    title="Household and State-wise Concern for Ability to Pay"
)

# Add annotations with smaller text for better readability
fig.update_traces(
    texttemplate='%{z:.1f}%',  # Keep percentage formatting with one decimal place
    textfont=dict(size=10)  # Reduce the font size to avoid clutter
)

# Update layout for a clearer view with bolded axis labels and title
fig.update_layout(
    xaxis_title="Household Type / State",
    yaxis_title="Level of Agreement",
    title={
        "text": "Concerns About Ability to Pay Across Household Types and States", 
        "x": 0.5, 
        "xanchor": "center"
    },
    coloraxis_colorbar=dict(
        title="Concern Level (%)"
    )
)

# Show the heatmap
fig.show()
fig.write_json("Concerns_about_cost")

#### **Summary:**
The heatmap visualization shows that many households, especially Young Households and Family Households, are highly concerned about their ability to pay during the renewable energy transition. This concern is particularly strong in VIC and NSW, where a significant percentage of respondents strongly or somewhat agree with the statement.